In [1]:
import scipp as sc
import scippneutron as scn
import numpy as np
import h5py as h5
from shutil import copyfile
import sys
from ipyfilechooser import FileChooser

In [2]:
def fix_nexus_scipp(infile, outfile):
    """
    Currently there are some tweaks to make file working in scipp. This function won't be necessary in the future
    """

    copyfile(infile, outfile)
    with h5.File(outfile, 'r+') as f:
         
        group = f['entry/instrument']
        for monitor_name in filter(lambda k: k.startswith('monitor'), group):
            monitor_group = group[monitor_name]
            monitor_event_group = monitor_group[f'{monitor_name}_events']
            for key in list(monitor_event_group):
                monitor_group[key] = monitor_event_group.pop(key)
            del monitor_group[f'{monitor_name}_events']


In [3]:
def fix_nexus_mantid(infile, outfile, didx):
    """
    Additional tweaks for loading into Mantid
    """

    copyfile(infile, outfile)
    with h5.File(outfile, 'r+') as f:
        
        f['entry/instrument/name'] = 'LARMOR'

        group_entry = f['entry']
        nx_class = group_entry.create_group('sample')
        nx_class.attrs["NX_class"] = 'NXsample'
       
        f['entry/larmor_detector_events'] = f['entry/instrument/larmor_detector/larmor_detector_events']
        
        #TODO: Adding detector_id. Not sure if this is 100% correct
        f['entry']['instrument']['monitor_1']['detector_id'] = np.array(didx+1)
        f['entry']['instrument']['monitor_2']['detector_id'] = np.array(didx+2)
        
        group = f['entry/instrument']
        for monitor_name in filter(lambda k: k.startswith('monitor'), group):
            monitor_group = group[monitor_name]
            monitor_event_group = monitor_group[f'{monitor_name}_events']
            for key in list(monitor_event_group):
                monitor_group[key] = monitor_event_group.pop(key)
            del monitor_group[f'{monitor_name}_events']


In [4]:
# Create and display a FileChooser widget
fc = FileChooser('/Users/wojciechpotrzebowski/SCIPP_SPACE/loki_detector_test/ess/docs/instruments/loki/Larmor_data')
display(fc)

FileChooser(path='/Users/wojciechpotrzebowski/SCIPP_SPACE/loki_detector_test/ess/docs/instruments/loki/Larmor_…

### Load and show data

In [12]:
input_file = fc.selected
fixed_file = f'{input_file[:-4]}_fixed.nxs'
fixed_mantid_file = f'{input_file[:-4]}_mantid.nxs'

In [13]:
fix_nexus_scipp(input_file, fixed_file)

In [14]:
data = scn.load_nexus(data_file=fixed_file)

## Tweaking file so it works in Mantid

In [15]:
fix_nexus_mantid(input_file, fixed_mantid_file, len(data.coords['position']))

In [ ]:
#fix_nexus_mantid(input_file, fixed_mantid_file,458752)

### Plot data and show how many events and pixels do we have

In [16]:
data

<scipp.DataArray>
Dimensions: Sizes[detector_id:458752, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1, 2, ..., 458751, 458752]
  position                  vector3              [m]  (detector_id)  [(-0.499023, 0.45326, 4.42333), (-0.497074, 0.45326, 4.42333), ..., (0.495061, -0.4437, 4.51745), (0.49701, -0.4437, 4.51745)]
  source_position           vector3              [m]  ()  [(0, 0, -25.3)]
  tof                         int32             [ns]  (tof [bin-edge])  [91, 100000046]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
Attributes:
  base_position             vector3              [m]  (detector_id)  [(0, -0.00114, -0.00767), (0.00194919, -0.00114, -0.00767), ..., (0.994085, -0.8981, 0.086447), (0.996034, -0.8981, 0.086447)]
  end_time                   string           [None]  ()  ["2032-03-02T09:05:46.690Z"]
  experiment_title           string           [None]  ()  ["My experiment"]
  monitor_1               DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1]
  tof                         int32             [ns]  (tof [bin-edge])  [7291, 99999036]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:10930751, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2022-03-02T21:05:46.106536566, 2022-03-02T21:05:46.106536566, ..., 2022-03-02T21:36:52.606456716, 2022-03-02T21:36:52.606456716]
  tof                         int32             [ns]  (event)  [14857386, 14983095, ..., 32689554, 33205110]
Data:
                            float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]

]

]
  monitor_2               DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [2]
  tof                         int32             [ns]  (tof [bin-edge])  [17388, 99998797]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:75397, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2022-03-02T21:05:46.106536566, 2022-03-02T21:05:46.106536566, ..., 2022-03-02T21:36:52.606456716, 2022-03-02T21:36:52.606456716]
  tof                         int32             [ns]  (event)  [22168104, 22493001, ..., 8892195, 11229652]
Data:
                            float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]

]

]
  position_transformations Variable           [None]  ()  [<scipp.Variable> ()  affine_transform3              [m]  [        1         0         0 -0.499023
        0         1         0    0.4544
  

In [ ]:
sc.max(data.coords['position'].fields.z) - sc.min(data.coords['position'].fields.z)

In [45]:
data.plot(norm='log')

In [59]:
data.coords['tof'][-1]

<scipp.Variable> ()      int32             [ns]  [100000046]

In [62]:
sc.collapse(data, keep='tof').plot()

AttributeError: 'dict' object has no attribute 'plot'

### Instrument view

In [46]:
scn.instrument_view(data, pixel_size=0.01, norm="log")

### Histograming data

In [77]:
data.coords['tof'][0].values

array(91, dtype=int32)

In [99]:
start_tof = data.coords['tof'][0].values
f'{start_tof}'

'91'

In [95]:
 len(data.bins.constituents['data'].values)

5209954

In [85]:
tof_edges = sc.linspace(dim='tof', start=data.coords['tof'][0].values, stop=data.coords['tof'][-1].values, num=100, unit='ns')
histogrammed = sc.histogram(data, bins=tof_edges)

In [88]:
histogrammed.sum('detector_id').plot()

In [57]:
z = data.coords['position'].fields.z
near = z.min()
far = z.max()
layer = ((z - near) * 800).astype('int32')
layer.unit = ''
layer.plot()

In [ ]:
scn.instrument_view(histoigramed, pixel_size=0.01)

## Sorting out monitors

In [ ]:
monitors = {
    'sample': {'incident': data.attrs["monitor_1"].value,
               'transmission': data.attrs["monitor_2"].value}
}

In [ ]:
monitors['sample']['incident'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)
monitors['sample']['incident'].coords['source_position'] = sc.vector(value = np.array([0, 0, -25.3]), unit = sc.units.m)
monitors['sample']['transmission'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)
monitors['sample']['transmission'].coords['source_position'] = sc.vector(value = np.array([0, 0, -25.3]), unit = sc.units.m)

In [36]:
tof_edges = sc.linspace(dim='tof', start=91, stop=100000046, num=400, unit='ns')
histogrammed_monitor1 = sc.histogram(monitors['sample']['incident'], bins=tof_edges)
histogrammed_monitor2 = sc.histogram(monitors['sample']['transmission'], bins=tof_edges)

NameError: name 'monitors' is not defined

In [ ]:
#sc.plot({'monitor1':histogrammed_monitor1.sum('detector_id'), 
#        'monitor2':histogrammed_monitor2.sum('detector_id')})

sc.plot({'monitor1':histogrammed_monitor1.sum('detector_id')})

In [ ]:
sc.plot({ 'monitor2':histogrammed_monitor2.sum('detector_id')})

In [ ]:
data.attrs["monitor_1"].value

In [ ]:
histoigramed.sum('detector_id').plot()

### Troubleshooting

In [ ]:
#path = 'Larmor_data'
#scn.load(filename=f'{path}/2022-02-23_1100_detector_id.nxs', mantid_alg='LoadEventNexus')

In [ ]:
binned_data = data.bins.sum()
scn.instrument_view(binned_data)
binned_data.plot()
binned_data
binned_data.sum('detector_id').plot(marker='.')